In [1]:
from torchdiffeq import odeint
from torchdiffeq import odeint_adjoint as odeint_adj
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
from random import sample
import time
import os
import copy

In [8]:
class ODEFunc(nn.Module):
    def __init__(self, twice_dim):
        super(ODEFunc, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(twice_dim, 50),
            nn.ReLU(),
            nn.Linear(50,50),
            nn.ReLU(),
            nn.Linear(50, twice_dim),
            
        )
        for m in self.net.modules():
            if isinstance(m,nn.Linear):
                nn.init.normal_(m.weight,mean=0,std=0.01)
                nn.init.constant_(m.bias,val=0)
        
    def forward(self, t, y):
        
        #print(y.shape)
        out = self.net(y)
        #out = torch.ones(y.shape)
        #out[:,0] = y[:,2]
        #out[:,1] = y[:,3]
        #out[:,2] = y[:,0]/(0.6666**2)
        #out[:,3] = y[:,1]/(0.3333**2)
        
        return out

In [9]:
class ODEFunc_exact_den(nn.Module):
    def __init__(self,ODE_model):
        super(ODEFunc_exact_den, self).__init__()
        self.net = ODE_model
        
    def forward(self, t, y):
#         start = time.time()
        #print("here")
        zero = torch.tensor([0.0]).view(-1,1)
        inp = torch.cat((zero,y[:,:-1]),dim=1)
        f = self.net(t,y[:,:-1])
        #print(time.time()-start)
        Jacobian = torch.autograd.functional.jacobian(self.net, (t,y[:,:-1]))
        #print(time.time()-start)
        
        Tr = torch.trace(Jacobian[1].view(twice_dim,twice_dim)).view(-1,1)
        
        out = torch.cat((f,-Tr),dim=1)
        return out

In [14]:
def Evaluate_H(f, q,p,M_inv):
    V_o,grad = f(q,p)
    H_o = V_o + 0.5*( np.matmul(p.reshape(1,-1),np.matmul(M_inv,p.reshape(-1,1)))   )
    H_o = H_o.reshape(-1)
    return H_o

In [15]:
def Neural_dynamics(f,q_temp,p_temp,M,M_inv,steps,eps,store=True):
    dim = q_temp.shape[0]
    
    batch_t = torch.linspace(0.0,eps*(steps-1),steps)
    #print(batch_t.shape)
    
    q_temp = torch.tensor(q_temp)
    p_temp = torch.tensor(p_temp)
    Zero = torch.tensor([0.0])
    init = torch.cat((q_temp,p_temp,Zero),dim=0).float()
    init = init.view(1,-1)
#     start = time.time()
    pred_y = odeint(func_mod, init, batch_t,method = 'rk4',options = dict(step_size=eps))
    #print(time.time()-start)
    #print(pred_y.shape)
    
    
    pred_y = pred_y.detach().numpy()
    p_y = pred_y[:,0,:-1]
    J = pred_y[:,0,-1]
    
    energies = []
    
    for i in range(p_y.shape[0]):
        q = p_y[i,:dim]
        p = p_y[i,dim:]
        curr_e = Evaluate_H(f,q,p,M_inv)
    
        energies.append(curr_e)
    
    energies = np.asarray(energies)
    #print(energies.shape)
    #print(p_y.shape)
    return (p_y[-1,:dim],p_y[-1,dim:]),J[-1],p_y,energies



In [21]:
def Neural_HMC(f,q0,num_samples,eps,steps,store=False):
    
    samples = []
    accept_rate = 0
    q = q0
    
    if store == True:
        stored_vals = np.zeros((num_samples,steps,twice_dim))
        energies = np.zeros((num_samples,steps))
        
    ind = 0
    M = np.identity(twice_dim//2)
    M_inv = np.linalg.inv(M)
    while len(samples) < num_samples:
        
        ####### Need to fix
        mean = np.zeros((twice_dim//2))
        cov = M
        p = np.random.multivariate_normal(mean, cov)
        
        q_temp = copy.deepcopy(q)
        p_temp = copy.deepcopy(p)
        
        (q_f,p_f),J,path,energy = Neural_dynamics(f,q_temp,p_temp,M,M_inv,steps,eps,store=True)
        stored_vals[ind,:,:] = np.asarray(path)
        energies[ind,:] = np.asarray(energy).reshape(-1)
        
        p_f *= -1
        q = q_f
        
        samples.append(q)
        
        if len(samples)%500 == 0 and len(samples)< 2000:
            if len(samples) == 500:
                recent_samples = np.asarray(samples)
            else:
                recent_samples = np.asarray(samples[-500:])
            if (np.abs(np.linalg.det(np.cov(recent_samples.T))) > 0.001):
                M_inv = np.cov(recent_samples.T)
                M = np.linalg.inv(M_inv)
            else:
                M = np.identity(twice_dim//2)
                M_inv = np.linalg.inv(M)
#             print(M_inv)
        ind+=1
             
    acceptance = accept_rate/num_samples
        
    return samples,stored_vals,energies, acceptance

In [22]:
def GaussianXD(q, p):
    dim = q.shape[0]
    sigma = np.ones(dim)
    V = 0.5*np.sum(q**2 / (sigma**2))
    grad = q/(sigma**2)
    return V, grad

def Shell2D(q, p):
    r0 = np.sqrt(2)
    sigma = 0.5
    r = np.sqrt(np.dot(q, q))
    V = abs(r-r0)/sigma
    
    if (r-r0) == 0 or r == 0:
        grad = np.array([0, 0])
    else:
        grad = (q*(r-r0)/(sigma*r*abs(r-r0)))
        
    return V, grad

def Wofe_Quapp(q, p):
    x = q[0]
    y = q[1]
    V = x**4 + y**4 - 2*x**2 - 4*y**2 + x*y + 0.3*x + 0.1*y
    grad = np.array([4*x**3 - 4*x + y + 0.3, 4*y**3 - 8*y + x + 0.1])
    return V, grad

In [26]:
# Wofe-Quapp
num_samples = [10000] #[100, 200, 500, 1000, 2000]
twice_dim = 4
init = np.random.randn(twice_dim//2)*2
traj_length = 10
traj_step_size = 0.1
num_of_runs = 5
potential = 'wofe_quapp'
potential_function = Wofe_Quapp
func = torch.load(f'{potential}/model')
func_mod = ODEFunc_exact_den(func)

for q in num_samples:
    for i in range(1, num_of_runs+1):
        start = time.perf_counter()
        samps,trajs,energies,acceptance = Neural_HMC(potential_function, init, q, traj_step_size, traj_length, store=True)

        if not os.path.exists(f'{potential}/ode_{q}'):
            os.makedirs(f'{potential}/ode_{q}')

        with open(f'{potential}/ode_{q}/{i}_hmc_samps.npy', 'wb') as f:
            np.save(f, samps)

        with open(f'{potential}/ode_{q}/{i}_info.npy', 'wb') as f:
            np.save(f, np.array([time.perf_counter() - start, acceptance]))
        print(q, i)


10000 1
10000 2
10000 3
10000 4
10000 5


In [27]:
# 2D Shell
num_samples = [10000]#[100, 200, 500, 1000, 2000]
twice_dim = 4
init = np.random.randn(twice_dim//2)*2
traj_length = 10
traj_step_size = 0.01
num_of_runs = 5
potential = '2d_shell'
potential_function = Shell2D
func = torch.load(f'{potential}/model')
func_mod = ODEFunc_exact_den(func)

for q in num_samples:
    for i in range(1, num_of_runs+1):
        start = time.perf_counter()
        samps,trajs,energies,acceptance = Neural_HMC(potential_function, init, q, traj_step_size, traj_length, store=True)

        if not os.path.exists(f'{potential}/ode_{q}'):
            os.makedirs(f'{potential}/ode_{q}')

        with open(f'{potential}/ode_{q}/{i}_hmc_samps.npy', 'wb') as f:
            np.save(f, samps)

        with open(f'{potential}/ode_{q}/{i}_info.npy', 'wb') as f:
            np.save(f, np.array([time.perf_counter() - start, acceptance]))
        print(q, i)


10000 1
10000 2
10000 3
10000 4
10000 5


In [ ]:
# Gaussian 10D
num_samples = [10000]#[100, 200, 500, 1000, 2000]
twice_dim = 20
init = np.random.randn(twice_dim//2)*2
traj_length = 10
traj_step_size = 0.1
num_of_runs = 5
potential = '10d_gaussian'
potential_function = GaussianXD
func = torch.load(f'{potential}/model')
func_mod = ODEFunc_exact_den(func)

for q in num_samples:
    for i in range(1, num_of_runs+1):
        start = time.perf_counter()
        samps,trajs,energies,acceptance = Neural_HMC(potential_function, init, q, traj_step_size, traj_length, store=True)

        if not os.path.exists(f'{potential}/ode_{q}'):
            os.makedirs(f'{potential}/ode_{q}')

        with open(f'{potential}/ode_{q}/{i}_hmc_samps.npy', 'wb') as f:
            np.save(f, samps)

        with open(f'{potential}/ode_{q}/{i}_info.npy', 'wb') as f:
            np.save(f, np.array([time.perf_counter() - start, acceptance]))
        print(q, i)
